In [109]:
import pickle
from tqdm import tqdm
import numpy as np
import os


In [110]:
def make_label(text_str):    # 从单词到label的转换, 如: 今天 ----> BE  麻辣肥牛: ---> BMME  的 ---> S
    text_len = len(text_str)
    if text_len == 1:
        return "S"
    return "B" + "M" * (text_len - 2) + "E"  # 除了开头是 B, 结尾是 E，中间都是Ｍ


In [111]:
def text_to_state(file="实训专题1/trainCorpus.txt"):  # 将原始的语料库转换为 对应的状态文件

    if os.path.exists("all_train_state.txt"):  # 如果存在该文件, 就直接退出
        return
    all_data = open(file, "r", encoding="gbk").read().split(
        "\n")  # 打开文件并按行切分到  all_data 中 , all_data  是一个list
    with open("all_train_state.txt", "w", encoding="utf-8") as f:  # 代开写入的文件
        # 逐行 遍历 , tqdm 是进度条提示 , data 是一篇文章, 有可能为空
        for d_index, data in tqdm(enumerate(all_data)):
            if data:  # 如果 data 不为空
                state_ = ""
                # 当前 文章按照空格切分, w是文章中的一个词语
                for w in data.split(" "):
                    if w:                                            # 如果 w 不为空
                        # 制作单个词语的label
                        state_ = state_ + make_label(w) + " "
                # 最后一行不要加 "\n" 其他行都加 "\n"
                if d_index != len(all_data) - 2:
                    state_ = state_.strip() + "\n"                   # 每一行都去掉 最后的空格
                # 写入文件, state_ 是一个字符串
                f.write(state_)


In [112]:
# 定义 HMM类, 其实最关键的就是三大矩阵
class HMM:
    def __init__(self, file_text="实训专题1/trainCorpus.txt", file_state="all_train_state.txt"):
        self.all_states = open(
            file_state, "r", encoding="utf-8").read().split("\n")   # 按行获取所有的状态
        self.all_texts = open(file_text, "r", encoding="gbk").read().split(
            "\n")     # 按行获取所有的文本
        # 给每个状态定义一个索引, 以后可以根据状态获取索引
        self.states_to_index = {"B": 0, "M": 1, "S": 2, "E": 3}
        # 根据索引获取对应状态
        self.index_to_states = ["B", "M", "S", "E"]
        # 状态长度 : 这里是4
        self.len_states = len(self.states_to_index)

        # 初始矩阵 : 1 * 4 , 对应的是 BMSE
        self.init_matrix = np.zeros((self.len_states))
        self.transfer_matrix = np.zeros(
            (self.len_states, self.len_states))            # 转移状态矩阵:  4 * 4 ,

        # 发射矩阵, 使用的 2级 字典嵌套
        # # 注意这里初始化了一个  total 键 , 存储当前状态出现的总次数, 为了后面的归一化使用
        self.emit_matrix = {"B": {"total": 0}, "M": {
            "total": 0}, "S": {"total": 0}, "E": {"total": 0}}

    # 计算 初始矩阵
    def cal_init_matrix(self, state):
        # BMSE 四种状态, 对应状态出现 1次 就 +1
        self.init_matrix[self.states_to_index[state[0]]] += 1

    # 计算转移矩阵
    def cal_transfer_matrix(self, states):
        # 状态转移 从当前状态转移到后一状态, 即 从 sta1 每一元素转移到 sta2 中
        sta_join = "".join(states)
        sta1 = sta_join[:-1]
        sta2 = sta_join[1:]
        for s1, s2 in zip(sta1, sta2):   # 同时遍历 s1 , s2
            self.transfer_matrix[self.states_to_index[s1],
                                 self.states_to_index[s2]] += 1

    # 计算发射矩阵
    def cal_emit_matrix(self, words, states):
        # 先把words 和 states 拼接起来再遍历, 因为中间有空格
        for word, state in zip("".join(words), "".join(states)):
            self.emit_matrix[state][word] = self.emit_matrix[state].get(
                word, 0) + 1
            # 注意这里多添加了一个  total 键 , 存储当前状态出现的总次数, 为了后面的归一化使用
            self.emit_matrix[state]["total"] += 1

    # 将矩阵归一化
    def normalize(self):
        self.init_matrix = self.init_matrix/np.sum(self.init_matrix)
        self.transfer_matrix = self.transfer_matrix / \
            np.sum(self.transfer_matrix, axis=1, keepdims=True)
        self.emit_matrix = {state: {word: t/word_times["total"]*1000 for word, t in word_times.items(
        ) if word != "total"} for state, word_times in self.emit_matrix.items()}

    # 训练开始, 其实就是3个矩阵的求解过程
    def train(self):
        if os.path.exists("three_matrix.pkl"):  # 如果已经存在参数了 就不训练了
            self.init_matrix, self.transfer_matrix, self.emit_matrix = pickle.load(
                open("three_matrix.pkl", "rb"))
            return
        # 按行读取文件, 调用3个矩阵的求解函数
        for words, states in tqdm(zip(self.all_texts, self.all_states)):
            words = words.split(" ")            # 在文件中 都是按照空格切分的
            states = states.split(" ")
            self.cal_init_matrix(states[0])     # 计算三大矩阵
            self.cal_transfer_matrix(states)
            self.cal_emit_matrix(words, states)
        self.normalize()      # 矩阵求完之后进行归一化
        pickle.dump([self.init_matrix, self.transfer_matrix,
                    self.emit_matrix], open("three_matrix.pkl", "wb"))  # 保存参数


In [113]:
def viterbi_cut(text, hmm):
    states = hmm.index_to_states
    emit_p = hmm.emit_matrix
    trans_p = hmm.transfer_matrix
    start_p = hmm.init_matrix
    V = [{}]
    path = {}
    for y in states:
        V[0][y] = start_p[hmm.states_to_index[y]] * emit_p[y].get(text[0], 0)
        path[y] = [y]
    for t in range(1, len(text)):
        V.append({})
        newpath = {}

        # 检验训练的发射概率矩阵中是否有该字
        neverSeen = text[t] not in emit_p['S'].keys() and \
            text[t] not in emit_p['M'].keys() and \
            text[t] not in emit_p['E'].keys() and \
            text[t] not in emit_p['B'].keys()
        for y in states:
            emitP = emit_p[y].get(
                text[t], 0) if not neverSeen else 1.0  # 设置未知字单独成词
            temp = []
            for y0 in states:
                if V[t - 1][y0] > 0:
                    temp.append(
                        (V[t - 1][y0] * trans_p[hmm.states_to_index[y0], hmm.states_to_index[y]] * emitP, y0))
            (prob, state) = max(temp, default=(hmm.init_matrix[2], 'S'))
            # (prob, state) = max([(V[t - 1][y0] * trans_p[hmm.states_to_index[y0],hmm.states_to_index[y]] * emitP, y0)  for y0 in states if V[t - 1][y0] > 0])
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath

    (prob, state) = max([(V[len(text) - 1][y], y) for y in states])  # 求最大概念的路径

    result = ""  # 拼接结果
    for t, s in zip(text, path[state]):
        result += t
        if s == "S" or s == "E":  # 如果是 S 或者 E 就在后面添加空格
            result += " "
    return result


In [114]:
text_to_state()

In [115]:
hmm = HMM()
hmm.train()

In [116]:
text = "深航客机攀枝花机场遇险：机腹轮胎均疑受损，跑道灯部分损坏"
viterbi_cut(text, hmm)

'深航 客机 攀枝 花机场 遇险 ： 机腹 轮胎 均 疑受 损， 跑 道灯 部分 损坏 '

In [117]:
def hmm_word_extract():
    # 读取文件
    corpus = []
    path = '实训专题1/flightnews.txt'
    content = ''
    for line in open(path, 'r', errors='ignore'):
        line = line.strip()
        content += line
    corpus.append(content)
    # 加载停用词
    stop_words = []
    path = '实训专题1/stopword.txt'
    for line in open(path, encoding='utf8'):
        line = line.strip()
        stop_words.append(line)
    # jieba分词
    split_words = []
    word_list = viterbi_cut(corpus[0], hmm)
    word_list = word_list.split(' ')
    for word in word_list:
        if word not in stop_words:
            split_words.append(word)
    # 提取前10个高频词
    dic = {}
    word_num = 10
    for word in split_words:
        dic[word] = dic.get(word, 0) + 1
    freq_word = sorted(dic.items(), key=lambda x: x[1],
                       reverse=True)[: word_num]
    print('样本：' + corpus[0])
    print('样本分词效果：' + '/ '.join(split_words))
    print('样本前10个高频词：' + str(freq_word))

In [118]:
hmm_word_extract()

样本：原标题：深航飞机攀枝花遇险官方报告：机长私自上跑道检查还迟报 来源：上游新闻上游新闻持续关注的“深圳航空ZH9247航班10月16日在四川攀枝花机场降落时发生跑道外接地不安全事件”有了最新进展——10月19日深夜，民航西南管理局通过媒体公布了初步调查报告。据上述报告披露，此次事件造成飞机机腹出现两处大小为20*12*3.2、45*5*贯穿（厘米）的损伤，客机面临“机毁人亡”风险；航班机长存在未经机场许可前往运行中的跑道、并在1小时后才向塔台报告等情况。民航业内人士告诉上游新闻记者，深圳航空当事机组降落阶段处置是否合规、损伤是否构成飞机报废条件等情况，还需要民航监管机构进一步调查，但飞机落地后机长未经许可前往跑道、落地一小时后才向塔台报告的行为，已经严重违反了民航相关安全规定。资深飞行员：距离机毁人亡，仅一步之遥上游新闻此前刊发《深航客机攀枝花机场遇险：机腹轮胎均疑受损，跑道灯部分损坏》、《深航回应航班攀枝花“航母机场”遇险被质疑避重就轻：只说扎胎不讲机腹受损》报道显示，10月16日，由西安咸阳飞往攀枝花保安营机场的深圳航空ZH9247航班在攀枝花机场落地时发生不安全事件，“落地阶段疑似机腹擦挂跑道监控天线，起落架提前接地，到位后检查轮胎有扎伤痕迹。”攀枝花机场事后对跑道、灯光等相关设施进行全面检查，“航向台监控天线被撞断、6个进近灯被撞坏”。深圳航空官方事后仅在微博简单回应称，该航班是飞机轮胎被扎导致后续航班取消，未披露更多的详细情况。深航的这一回应，被舆论质疑为避重就轻、选择性披露。10月19日深夜，民航西南局通过四川当地媒体《川观新闻》，披露了10月16日深航ZH9247航班在攀枝花机场跑道外接地事件的初步调查报告。报告认定该航班在降落时，出现了“跑道外接地的不安全事件”。资料显示，“跑道外接地”是指客机在落地时没有在标准硬化跑道上落地，在跑道外同地面发生接触，使飞机有可能陷入跑道外的地面，容易造成飞机机身倾斜、撕裂、起落架设备受损等严重后果。国内某航空公司资深飞行员赖先生对上游新闻记者表示，在攀枝花这样的山顶机场发生跑道外接地事件，“距离机毁人亡，仅一步之遥。”机腹现两处损伤，最严重可致飞机报废初步调查报告披露，当天航班上共计99人，含3名飞行机组人员、6名客舱机组人员、1名跟机机务和89名旅客。初步调查报告也披露了10月16日执飞航班的B-8667空中

In [119]:
import jieba
def jieba1_word_extract():
    # 读取文件
    corpus = []
    path = '实训专题1/flightnews.txt'
    content = ''
    for line in open(path, 'r', errors='ignore'):
        line = line.strip()
        content += line
    corpus.append(content)
    # 加载停用词
    stop_words = []
    path = '实训专题1/stopword.txt'
    for line in open(path, encoding='utf8'):
        line = line.strip()
        stop_words.append(line)
        # jieba分词
    split_words = []
    word_list = jieba.cut(corpus[0])
    for word in word_list:
        if word not in stop_words:
            split_words.append(word)
    # 提取前10个高频词
    dic = {}
    word_num = 10
    for word in split_words:
        dic[word] = dic.get(word, 0) + 1
    freq_word = sorted(dic.items(), key = lambda x: x[1],
                       reverse=True) [: word_num]
    print('样本：' + corpus[0])
    print('样本分词效果：' + '/ '.join(split_words))
    print('样本前10个高频词：' + str(freq_word))

In [120]:
jieba1_word_extract()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.642 seconds.
Prefix dict has been built successfully.


样本：原标题：深航飞机攀枝花遇险官方报告：机长私自上跑道检查还迟报 来源：上游新闻上游新闻持续关注的“深圳航空ZH9247航班10月16日在四川攀枝花机场降落时发生跑道外接地不安全事件”有了最新进展——10月19日深夜，民航西南管理局通过媒体公布了初步调查报告。据上述报告披露，此次事件造成飞机机腹出现两处大小为20*12*3.2、45*5*贯穿（厘米）的损伤，客机面临“机毁人亡”风险；航班机长存在未经机场许可前往运行中的跑道、并在1小时后才向塔台报告等情况。民航业内人士告诉上游新闻记者，深圳航空当事机组降落阶段处置是否合规、损伤是否构成飞机报废条件等情况，还需要民航监管机构进一步调查，但飞机落地后机长未经许可前往跑道、落地一小时后才向塔台报告的行为，已经严重违反了民航相关安全规定。资深飞行员：距离机毁人亡，仅一步之遥上游新闻此前刊发《深航客机攀枝花机场遇险：机腹轮胎均疑受损，跑道灯部分损坏》、《深航回应航班攀枝花“航母机场”遇险被质疑避重就轻：只说扎胎不讲机腹受损》报道显示，10月16日，由西安咸阳飞往攀枝花保安营机场的深圳航空ZH9247航班在攀枝花机场落地时发生不安全事件，“落地阶段疑似机腹擦挂跑道监控天线，起落架提前接地，到位后检查轮胎有扎伤痕迹。”攀枝花机场事后对跑道、灯光等相关设施进行全面检查，“航向台监控天线被撞断、6个进近灯被撞坏”。深圳航空官方事后仅在微博简单回应称，该航班是飞机轮胎被扎导致后续航班取消，未披露更多的详细情况。深航的这一回应，被舆论质疑为避重就轻、选择性披露。10月19日深夜，民航西南局通过四川当地媒体《川观新闻》，披露了10月16日深航ZH9247航班在攀枝花机场跑道外接地事件的初步调查报告。报告认定该航班在降落时，出现了“跑道外接地的不安全事件”。资料显示，“跑道外接地”是指客机在落地时没有在标准硬化跑道上落地，在跑道外同地面发生接触，使飞机有可能陷入跑道外的地面，容易造成飞机机身倾斜、撕裂、起落架设备受损等严重后果。国内某航空公司资深飞行员赖先生对上游新闻记者表示，在攀枝花这样的山顶机场发生跑道外接地事件，“距离机毁人亡，仅一步之遥。”机腹现两处损伤，最严重可致飞机报废初步调查报告披露，当天航班上共计99人，含3名飞行机组人员、6名客舱机组人员、1名跟机机务和89名旅客。初步调查报告也披露了10月16日执飞航班的B-8667空中